# Lab: Deploying an end point using Amazon SageMaker

Amazon SageMaker is composed of 5 main services: hosted notebooks, built-in algorithms, model training, hyperparameter tuning, and model hosting. They all fit in nicely together and can also be used independendly as needed. In this lab we explore how to deploy and host a model so it can be used in production to evaluate novel inputs. THe hosted endpoints can either be used on single examples or in batch mode over a larger number of examples.

In this lab we explore three different ways of deploying the endpoint:

1. Using a model trained with Amazon SageMaker.
2. From a model trained elsewhere with artifacts stored in Amazon S3.
3. From a model developed with a custom docker container.

In [1]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## <span style="color:blue">Section 1: Deploying a model trained with Amazon SageMaker</span>

To start with a model is trained with Amazon SageMaker and then directly deployed to an endpoint.

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This part of the lab will show how to train and test an MNIST model on SageMaker using MXNet and the Gluon API.

### Download training and test data

In [2]:
gluon.data.vision.MNIST('./data/train', train=True)
gluon.data.vision.MNIST('./data/test', train=False)

### Uploading the data

We use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the location -- we will use this later when we start the training job.

In [3]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')

### Implement the training function

We need to provide a training script that can run on the SageMaker platform. The training scripts are essentially the same as one you would write for local training, except that you need to provide a `train` function. When SageMaker calls your function, it will pass in arguments that describe the training environment. Check the script below to see how this works.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [4]:
!cat 'mnist.py'

from __future__ import print_function

import logging
import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn
import numpy as np
import json
import time


logging.basicConfig(level=logging.DEBUG)

# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #


def train(current_host, channel_input_dirs, hyperparameters, hosts, num_gpus):
    # SageMaker passes num_cpus, num_gpus and other args we can use to tailor training to
    # the current container environment, but here we just use simple cpu context.
    ctx = mx.cpu()

    # retrieve the hyperparameters we set in notebook (with some defaults)
    batch_size = hyperparameters.get('batch_size', 100)
    epochs = hyperparameters.get('epochs', 10)
    learning_rate = hyperparameters.get('learning_rate', 0.1)
    momentum = hyperparameters.get('momentum', 0.9)
    log_interval = 

### Run the training script on SageMaker

The ```MXNet``` class allows us to run our training function on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on a single c4.xlarge instance. 

In [5]:
m = MXNet("mnist.py", 
          role=role, 
          train_instance_count=1, 
          train_instance_type="ml.c4.xlarge",
          hyperparameters={'batch_size': 100, 
                         'epochs': 20, 
                         'learning_rate': 0.1, 
                         'momentum': 0.9, 
                         'log_interval': 100})

After we've constructed our `MXNet` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [6]:
m.fit(inputs)

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-07-20-08-45-48-381


...............
2018-07-20 08:48:02,723 INFO - root - running container entrypoint
2018-07-20 08:48:02,724 INFO - root - starting train task
2018-07-20 08:48:02,729 INFO - container_support.training - Training starting
2018-07-20 08:48:05,053 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 0, 'channels': {u'training': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'hosts': [u'algo-1'], u'network_interface_name': u'ethwe', u'current_host': u'algo-1'}, 'user_script_name': u'mnist.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'training': u'/opt/ml/input/data/training'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'output_dir': '/opt/ml/output', 'model_dir': '/opt/ml/model', 'hyperparameters': {u'sagemaker_program': u'mnist.py', u'learning_rate': 0.1, u'batch_size': 100, u'epochs':

[Epoch 10] Validation: accuracy=0.975800
[Epoch 11 Batch 100] Training: accuracy=0.994653, 4401.758897 samples/s
[Epoch 11 Batch 200] Training: accuracy=0.992935, 3217.280314 samples/s
[Epoch 11 Batch 300] Training: accuracy=0.993223, 4493.576173 samples/s
[Epoch 11 Batch 400] Training: accuracy=0.993167, 4208.064370 samples/s
[Epoch 11 Batch 500] Training: accuracy=0.993054, 3861.162868 samples/s
[Epoch 11] Training: accuracy=0.993267
[Epoch 11] Validation: accuracy=0.978400
[Epoch 12 Batch 100] Training: accuracy=0.992673, 4674.204585 samples/s
[Epoch 12 Batch 200] Training: accuracy=0.992289, 4320.773026 samples/s
[Epoch 12 Batch 300] Training: accuracy=0.992425, 4328.085111 samples/s
[Epoch 12 Batch 400] Training: accuracy=0.992843, 4398.896685 samples/s
[Epoch 12 Batch 500] Training: accuracy=0.993014, 4363.158223 samples/s
[Epoch 12] Training: accuracy=0.992600
[Epoch 12] Validation: accuracy=0.976100
[Epoch 13 Batch 100] Training: accuracy=0.994851, 4500.422756 samples/s
[Epoch 

After training, we use the MXNet object to build and deploy an MXNetPredictor object. This creates a SageMaker endpoint that we can use to perform inference. 

This allows us to perform inference on json encoded multi-dimensional arrays.

The deploy method does the following in order:

1. Creates an Amazon SageMaker model by calling the CreateModel API. The model that you create in Amazon SageMaker holds information such as location of the model artifacts and the inference code image.

2. Creates an endpoint configuration by calling the CreateEndpointConfig API. This configuration holds necessary information including the name of the model (which was created in the preceding step) and the resource configuration (the type and number of ML compute instances to launch for hosting).

3. Creates the endpoint by calling the CreateEndpoint API and specifying the endpoint configuration created in the preceding step. Amazon SageMaker launches ML compute instances as specified in the endpoint configuration, and deploys the model on them.

In [7]:
predictor = m.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-07-20-08-45-48-381
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-07-20-08-45-48-381


---------------------------------------------------------------!

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the mxnet predictor. 

In [8]:
from IPython.display import HTML
HTML(open("input.html").read())

The predictor runs inference on our input data and returns the predicted digit (as a float value, so we convert to int for display).

In [11]:
response = predictor.predict(data)
print int(response)

2


### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [12]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: sagemaker-mxnet-2018-07-20-08-45-48-381


## <span style="color:blue"> Section 2: Deploying from S3

Amazon SageMaker saves and stores the artifacts in S3. In the case where you have already trained a model elsewhere, it is relatively straightforward to upload the model artifacts to S3 and deploy the model using Amazon SageMaker.

In this case the model trained in the previous section will be used. We will create a model using the [sagemaker.mxnet.model.MXNetModel](http://sagemaker.readthedocs.io/en/latest/sagemaker.mxnet.html#mxnet-model) function, passing the location in S3 with zipped and archived copy of the model artifacts, and then deploy it using S3.

### Locate the model artifacts in S3

The model in the previous section saved the artifacts including the parameters and model declaration in S3. This will be in the default bucket which SageMaker has access to used by the notebook. Locate this bucket and scroll down to find the latest folder that was added. Inside this folder under 'output' you will find a 'model.tar.gz' file. Download this, unpack it, and have a look at the contents. Here is and example path:

bucket-name/sagemaker-mxnet-2018-07-20-08-45-48-381/output/model.tar.gz

Two files are contained:

* model.json - The declaration of the network
* model.params - The trained parameters of the model

#### Copy the model artifacts to a new location

Choose a location in S3 and copy the model.tar.gz into this folder. Take a note of the bucket name and the path to the file within the bucket.

#### Load the model using MXNetModel

Load model from S3 using [sagemaker.mxnet.model.MXNetModel](http://sagemaker.readthedocs.io/en/latest/sagemaker.mxnet.html#mxnet-model). Enter below under model_data the bucket and path of the file noted in the previous step.

In [31]:
from sagemaker.mxnet.model import MXNetModel

model_data = 's3://sagemaker-eu-west-1-987551451182/sagemaker-mxnet-2018-07-20-08-45-48-381/output/model.tar.gz'

m2 = MXNetModel(model_data, role, "mnist.py")

In [22]:
predictor = m2.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-07-20-08-45-48-381


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Cannot create already existing model "arn:aws:sagemaker:eu-west-1:987551451182:model/sagemaker-mxnet-2018-07-20-08-45-48-381".

This will fail. This is because you already have a model set up with the same configuration. How do you think it knows that it is the same model? Go to the Amazon SageMaker console and find the model under Inference -> Models and copy the arn of the model. Use it in the code below to delete the model using the last part of the arn after the forward slash:

In [44]:
import boto3
sm = boto3.client('sagemaker')
sm.delete_model(ModelName='sagemaker-mxnet-2018-07-20-12-33-35-466')

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 20 Jul 2018 12:45:37 GMT',
   'x-amzn-requestid': '14b5adab-9e3d-4299-a6f6-5b14e9a17d47'},
  'HTTPStatusCode': 200,
  'RequestId': '14b5adab-9e3d-4299-a6f6-5b14e9a17d47',
  'RetryAttempts': 0}}

Deleting the model does not delete the artifacts stored in S3. Now try to create the model again:

In [38]:
p2 = m2.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-07-20-12-37-14-916
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-07-20-12-37-14-916


---------------------------------------------------!

Use the same input you drew in the box above and score against this new endpoint:

In [40]:
response = p2.predict(data)
print int(response)

2


### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [41]:
sagemaker.Session().delete_endpoint(p2.endpoint)

INFO:sagemaker:Deleting endpoint with name: sagemaker-mxnet-2018-07-20-12-37-14-916


## <span style="color:blue"> Section 3: Deploying using a customer Docker container.

Deploying using a customer docker container. This is covered in a separate lab [scikitlearn bring your own](../scikit_bring_your_own/scikit_bring_your_own.ipynb) where a Docker container is built and deployed to ECS, trained, and then a SageMaker endpoint is deployed.